In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib as plt
import basedosdados as bd
sns.set_theme(style="darkgrid")

In [ ]:
#conda update -n base -c defaults conda

In [ ]:

# Instalar o pacote da basedosdados, caso seja a primeira vez a rodar o script
# ! pip install basedosdados

In [3]:
# Para carregar e salvar os dados 

query = """

SELECT ano, mes, sigla_uf, saldo_movimentacao, id_municipio, cnae_2_secao, categoria, cnae_2_subclasse, cbo_2002, grau_instrucao, sexo, salario_mensal, indicador_trabalho_intermitente,
indicador_trabalho_parcial,   FROM `basedosdados.br_me_caged.microdados_movimentacao` WHERE sigla_uf = "MA" AND cnae_2_subclasse  IN (
      --esssas duas subclasses são da divisão 50 TRANSPORTE AQUAVIÁRIO
      '5231101', --Administração da infraestrutura portuária
      '5231102', --Atividades do operador portuário
      --esssas outras são novas
      '5011401', --Transporte marítimo de cabotagem - carga
      '5011402', --Transporte marítimo de cabotagem - passageiros
      '5021101', --Transporte por navegação interior de carga, municipal, exceto travessia
      '5021102', --Transporte por navegação interior de carga, intermunicipal, interestadual e internacional, exceto travessia
      '5022001', --Transporte por navegação interior de passageiros em linhas regulares, municipal, exceto travessia
      '5022002', --Transporte por navegação interior de passageiros em linhas regulares, intermunicipal, interestadual e internacional, exceto travessia
      '5030101', --Navegação de apoio marítimo
      '5030102', --Navegação de apoio portuário
      '5030103', --Serviço de rebocadores e empurradores
      '5091201', --Transporte por navegação de travessia, municipal
      '5091202', --Transporte por navegação de travessia intermunicipal, interestadual e internacional
      '5099801', --Transporte aquaviário para passeios turísticos
      '5099899' --Outros transportes aquaviários não especificados anteriormente
)


""" 

dados = bd.read_sql(query, billing_project_id="caged-368713")   # Cada usuario deve setar o proprio project_id 

Downloading: 100%|██████████| 2709/2709 [00:00<00:00, 3888.87rows/s]


In [4]:
# salvar os dados em .csv 
dados.to_csv("dados.csv", encoding = 'utf-8')

In [5]:
dados = pd.read_csv("dados.csv")

In [6]:
# verificar os dados 
dados.head(5)

,Unnamed: 0,ano,mes,sigla_uf,saldo_movimentacao,id_municipio,cnae_2_secao,categoria,cnae_2_subclasse,cbo_2002,grau_instrucao,sexo,salario_mensal,indicador_trabalho_intermitente,indicador_trabalho_parcial
0,0,2020,2,MA,1,2111300,H,101,5231102,715135,7,1,NaN,0,0
1,1,2020,2,MA,1,2111300,H,101,5231102,411010,9,3,NaN,0,0
2,2,2020,2,MA,1,2111300,H,101,5030102,215115,7,1,NaN,0,0
3,3,2020,2,MA,-1,2112001,H,999,5091202,421125,5,3,NaN,0,0
4,4,2020,2,MA,-1,2111300,H,101,5231102,784205,5,1,NaN,0,0


In [9]:
dados.shape

(2709, 15)

In [10]:
dados['ano_mes'] = dados['ano'].map(str) + '-' + dados['mes'].map(str) + '-01'
dados.head(5)

,Unnamed: 0,ano,mes,sigla_uf,saldo_movimentacao,id_municipio,cnae_2_secao,categoria,cnae_2_subclasse,cbo_2002,grau_instrucao,sexo,salario_mensal,indicador_trabalho_intermitente,indicador_trabalho_parcial,ano_mes
0,0,2020,2,MA,1,2111300,H,101,5231102,715135,7,1,NaN,0,0,2020-2-01
1,1,2020,2,MA,1,2111300,H,101,5231102,411010,9,3,NaN,0,0,2020-2-01
2,2,2020,2,MA,1,2111300,H,101,5030102,215115,7,1,NaN,0,0,2020-2-01
3,3,2020,2,MA,-1,2112001,H,999,5091202,421125,5,3,NaN,0,0,2020-2-01
4,4,2020,2,MA,-1,2111300,H,101,5231102,784205,5,1,NaN,0,0,2020-2-01


In [11]:
from datetime import datetime
import locale

locale.setlocale(locale.LC_ALL, 'pt_BR')
formato = "%Y-%m-%d"
dados.ano_mes = dados.ano_mes.apply(lambda linha: datetime.strptime(linha, formato))

In [12]:
dados.ano_mes

0      2020-02-01
1      2020-02-01
2      2020-02-01
3      2020-02-01
4      2020-02-01
          ...    
2704   2022-02-01
2705   2022-02-01
2706   2022-02-01
2707   2022-02-01
2708   2022-02-01
Name: ano_mes, Length: 2709, dtype: datetime64[ns]

In [20]:
# Vamos fazer o label encoding. 
# criar dicionário para recodificar variavel 
# https://www.sharpsightlabs.com/blog/recode-categorical-variable-python/ 
# dados.assign(teste  = dados.sexo.map(sexo_mapping)) Usar essa abordagem para testar antes de salvar diretamente no dataframe

sexo_mapping = {1:'Masculino',
                3:'Feminino'
                    }

dados['sexo'] = dados['sexo'].map(sexo_mapping) 

dados.sexo.value_counts()


Series([], Name: sexo, dtype: int64)

In [ ]:
dados.head(3)

In [ ]:
regiao_mapping = {1:'Norte',
                2:'Nordeste', 
                3:'Sudeste', 
                4:'Sul', 
                5:'Centro-Oeste', 
                9: 'nan'
                    }




In [ ]:
uf_mapping = {
    11:'Rondonia',
12:'Acre',
13:'Amazonas',
14:'Roraima',
15:'Para',
16:'Amapa',
17:'Tocantins',
21:'Maranhao',
22:'Piaui',
23:'Ceara',
24:'Rio-Grande-do-Norte',
25:'Paraiba',
26:'Pernambuco',
27:'Alagoas',
28:'Sergipe',
29:'Bahia',
31:'Minas-gerais',
32:'Espirito-Santo',
33:'Rio-de-Janeiro', 
35:'Sao-Paulo',
41:'Parana',
42:'Santa Catarina',
43:'Rio Grande do Sul',
50:'Mato Grosso do Sul',
51:'Mato Grosso',
52:'Goias',
53:'Distrito-Federal',
99: 'nan'

}

In [ ]:
grauinstrucao_mapping = {1:'Analfabeto',
2:'Até 5ª Incompleto',
3:'5ª Completo Fundamental',
4:'6ª a 9ª Fundamental',
5:'Fundamental Completo',
6:'Médio Incompleto',
7:'Médio Completo',
8:'Superior Incompleto',
9:'Superior Completo',
10:'Mestrado',
11:'Doutorado',
80:'Pós-Graduação completa',
99:'Não Identificado'

}

dados['grau_instrucao'] = dados['grau_instrucao'].map(grauinstrucao_mapping)

dados.grau_instrucao.value_counts()

Leitura dos dados salvos no Drive

In [ ]:
descending_order = dados["grau_instrucao"].value_counts().sort_values(ascending=False).index

ax = sns.countplot(data=dados, y="grau_instrucao", order=descending_order)



In [ ]:
parcial_mapping = {1:'Sim', 
                0:'Nao', 
                9:'nan'
                    }


dados['indicador_trabalho_parcial'] = dados['indicador_trabalho_parcial'].map(parcial_mapping)

dados.indicador_trabalho_parcial.value_counts()




In [ ]:
ax = sns.countplot(x="indicador_trabalho_parcial", data=dados)

#Adicionar label com casa decimal. Verificar aqui: https://stackoverflow.com/questions/70515542/adding-comma-to-bar-labels 

#ax.bar_label(ax.containers[0], label_type='edge')
container = ax.containers[0]
ax.bar_label(container, labels=[f'{x:,.0f}' for x in container.datavalues])

#https://www.kaggle.com/code/slamnz/seaborn-tutorial-count-plots
#https://medium.com/swlh/quick-guide-to-labelling-data-for-common-seaborn-plots-736e10bf14a9 

In [ ]:
intermitente_mapping = {1:'Sim', 
                0:'Nao', 
                9:'nan'
                    }

dados['indicador_trabalho_intermitente'] = dados['indicador_trabalho_intermitente'].map(intermitente_mapping)

dados.indicador_trabalho_intermitente.value_counts()

In [ ]:
plot_intermitente = sns.countplot(x="indicador_trabalho_intermitente", data=dados)

plot_intermitente.bar_label(plot_intermitente.containers[0], label_type='edge')

In [ ]:
from matplotlib.pyplot import xticks

lineplot = sns.lineplot(data=dados, x="ano_mes", y="indicador_trabalho_intermitente", hue = "sexo")
#xticks(rotation=40)


lineplot.set_xticklabels(lineplot.get_xticklabels(),rotation=30)
lineplot.set_xlabel("Mês")
lineplot.set_ylabel("Trabalho Intermitente")


#https://stackoverflow.com/questions/44954123/rotate-xtick-labels-in-seaborn-boxplot 
#https://stackoverflow.com/questions/45201514/how-to-edit-a-seaborn-legend-title-and-labels-for-figure-level-functions


In [ ]:
pip install tabulate
#https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd

In [ ]:
from tabulate import tabulate


In [ ]:
intermitente = dados.indicador_trabalho_intermitente.value_counts()



In [ ]:
print(tabulate(dados, tablefmt='fancy_grid'))


In [ ]:
# secao cnae

